In [67]:
import tensorflow as tf

import os
import pathlib
import time
import datetime

import numpy as np

from matplotlib import pyplot as plt
from IPython import display

import numpy as np
from sklearn.model_selection import GridSearchCV

import sys

sys.path.append('/home/natanael/deblurring_images')
from data import Dataset
from models.pixpix import Pix2Pix

os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'


In [68]:
# Get the list of physical GPU devices
physical_devices = tf.config.list_physical_devices('GPU')

# Set memory growth for each physical GPU (if available)
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [ ]:
dataset = Dataset(buffer_size=400, batch_size=1 )

train_dataset = dataset.train
test_dataset = dataset.test

In [ ]:
LAMBDA = 100
loss_object = tf.keras.losses.BinaryCrossentropy(from_logits=True)
mse_loss = tf.keras.losses.MeanSquaredError()

In [ ]:
def generator_loss(disc_generated_output, gen_output, target):
    gan_loss = loss_object(tf.ones_like(disc_generated_output), disc_generated_output)
    
    # Mean absolute error
    l1_loss = tf.reduce_mean(tf.abs(target - gen_output))
    
    total_gen_loss = gan_loss + (LAMBDA * l1_loss)
    
    return total_gen_loss

In [ ]:
def generator_loss_mse(disc_generated_output, gen_output, target):
    return mse_loss(target, gen_output)

In [ ]:
def discriminator_loss(disc_real_output, disc_generated_output):
  real_loss = loss_object(tf.ones_like(disc_real_output), disc_real_output)

  generated_loss = loss_object(tf.zeros_like(disc_generated_output), disc_generated_output)

  total_disc_loss = real_loss + generated_loss

  return total_disc_loss

In [ ]:
!mkdir ./train_checkpoints_mse_loss

In [ ]:
def score(y_true, y_pred):
    """Computes the accuracy of the model.

    Args:
      y_true: The ground truth labels.
      y_pred: The predicted labels.

    Returns:
      The accuracy of the model.
    """

    correct = tf.equal(y_true, y_pred)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    return accuracy

In [ ]:
global step
step = 0

In [ ]:
def generate_images(model, test_input, tar):
    prediction = model(test_input, training=True)

    global step

    score_model = score(tar, prediction)
    score_mse = mse_loss(tar, prediction)
    score_ssim = tf.image.ssim(tar, prediction, max_val=1.0, filter_size=11,
                          filter_sigma=1.5, k1=0.01, k2=0.03)
    score_psnr = tf.image.psnr(tar, prediction, max_val=1.0)

    with summary_writer.as_default():
        tf.summary.scalar('acuracy', score_model, step=step//1000)
        tf.summary.scalar('mse', score_mse, step=step//1000)
        tf.summary.scalar('ssim', score_ssim[0], step=step//1000)
        tf.summary.scalar('psnr', score_psnr[0], step=step//1000)

    step += 1000
 

In [ ]:
!mkdir ./logs_mse_loss

In [ ]:
log_dir="./logs_mse_loss/"

summary_writer = tf.summary.create_file_writer(
  log_dir + "fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))

In [ ]:
def log_saved(gen_loss_total=None,
                disc_loss_total=None,
                step=None):
    
    with summary_writer.as_default():
        tf.summary.scalar('disc_loss_total', disc_loss_total, step=step//1000)
        tf.summary.scalar('gen_loss_total', gen_loss_total, step=step//1000)

In [ ]:
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
checkpoint_dir = './train_checkpoints_mse_loss'

pix = Pix2Pix()

pix.compile(
  otimazer_generator=generator_optimizer,
  otimazer_discriminator=discriminator_optimizer, 
  loss_generator=generator_loss_mse, 
  loss_discriminator=discriminator_loss, 
  checkpoint_dir=checkpoint_dir
)

In [ ]:
pix.fit(train_dataset, test_dataset, steps=50000, view_callback=generate_images, log_saved=log_saved)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /home/natanael/deblurring_images/notebooks/logs_mse_loss/fit

In [ ]:
def generate_images_view(model, test_input, tar):
  prediction = model(test_input, training=True)
  plt.figure(figsize=(15, 15))

  display_list = [test_input[0], tar[0], prediction[0]]
  title = ['Input Image', 'Ground Truth', 'Predicted Image']

  for i in range(3):
    plt.subplot(1, 3, i+1)
    plt.title(title[i])
    # Getting the pixel values in the [0, 1] range to plot.
    plt.imshow(display_list[i] * 0.5 + 0.5)
    plt.axis('off')
  plt.show()

In [ ]:
# Run the trained model on a few examples from the test set
for inp, tar in test_dataset.take(5):
  generate_images_view(pix.generator, inp, tar)